In [4]:
import sys

parent_dir = 'Predict-Future-Sales'
p_sub = sys.path[0]

ride = ''
for path in p_sub.split('/'):
    if path != parent_dir:
        ride = ride + path + '/'
    else:
        ride = ride + path + '/'
        break
sys.path[0] = ride

import numpy as np
import pandas as pd
import torch

from module.lino_module.preprocess import mode_of_freq

from typing import Tuple, Optional, Union
from numpy import ndarray
from pandas import DataFrame, Series, DatetimeIndex
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

### Time Delay Embedding に対応させた曜日と月時情報をd_modelにconcatしたデータセットを出力する

In [5]:
from module.lino_module.preprocess import tde_dataset_wm

data = pd.read_csv('../data/sales_train.csv')
data = mode_of_freq(data).item_cnt_day
demo = np.arange(len(data))
ds = pd.Series(demo , index=data.index)
ds

date
2013-01-01       0
2013-01-02       1
2013-01-03       2
2013-01-04       3
2013-01-05       4
              ... 
2015-10-27    1029
2015-10-28    1030
2015-10-29    1031
2015-10-30    1032
2015-10-31    1033
Freq: D, Length: 1034, dtype: int64

In [11]:
d_model = 5
kwrgs ={'data': ds,
        'seq': 7,
        'd_model': d_model,
        'dilation': 3,
        'src_tgt_seq': (4, 3),
        'step_num': 2,
        'batch_size': 64,
        'scaler': None,
        'daily': True,
        'weekday': True,
        'weekly': True,
        'monthly': True,
        'train_rate': 0.9 }

train, test = tde_dataset_wm(**kwrgs)
src, tgt, y = next(iter(train))
src.shape, tgt.shape, y.shape

(torch.Size([64, 4, 25]), torch.Size([64, 3, 25]), torch.Size([64, 3]))

In [12]:
index = 0
center = 40
print(' src_data '.center(center, '-'))
print(src[index][:, :d_model])

print(' daily '.center(center, '-'))
print(src[index][:, d_model:d_model*2])
print(' weekday '.center(center, '-'))
print(src[index][:, d_model*2:d_model*3])
print(' weekly '.center(center, '-'))
print(src[index][:, d_model*3:d_model*4])
print(' monthly '.center(center, '-'))
print(src[index][:, d_model*4:d_model*5])

--------------- src_data ---------------
tensor([[ 0.,  4.,  8., 12., 16.],
        [ 1.,  5.,  9., 13., 17.],
        [ 2.,  6., 10., 14., 18.],
        [ 3.,  7., 11., 15., 19.]])
---------------- daily -----------------
tensor([[0.0000, 0.1290, 0.2581, 0.3871, 0.5161],
        [0.0323, 0.1613, 0.2903, 0.4194, 0.5484],
        [0.0645, 0.1935, 0.3226, 0.4516, 0.5806],
        [0.0968, 0.2258, 0.3548, 0.4839, 0.6129]])
--------------- weekday ----------------
tensor([[0.1667, 0.8333, 0.3333, 1.0000, 0.5000],
        [0.3333, 1.0000, 0.5000, 0.0000, 0.6667],
        [0.5000, 0.0000, 0.6667, 0.1667, 0.8333],
        [0.6667, 0.1667, 0.8333, 0.3333, 1.0000]])
---------------- weekly ----------------
tensor([[0.0000, 0.0000, 0.0227, 0.0227, 0.0455],
        [0.0000, 0.0000, 0.0227, 0.0455, 0.0455],
        [0.0000, 0.0227, 0.0227, 0.0455, 0.0455],
        [0.0000, 0.0227, 0.0227, 0.0455, 0.0455]])
--------------- monthly ----------------
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 

In [5]:
print(' tgt_data '.center(center, '-'))
print(tgt[index][:, :d_model])
print(' daily '.center(center, '-'))
print(tgt[index][:, d_model:d_model*2])
print(' weekday '.center(center, '-'))
print(tgt[index][:, d_model*2:d_model*3])
print(' weekly '.center(center, '-'))
print(tgt[index][:, d_model*3:d_model*4])
print(' monthly '.center(center, '-'))
print(tgt[index][:, d_model*4:d_model*5])
print(' y '.center(center, '-'))
print(y[index])

--------------- tgt_data ---------------
tensor([[ 4.,  6.,  8., 10., 12.],
        [ 5.,  7.,  9., 11., 13.],
        [ 6.,  8., 10., 12., 14.]])
---------------- daily -----------------
tensor([[0.1290, 0.1935, 0.2581, 0.3226, 0.3871],
        [0.1613, 0.2258, 0.2903, 0.3548, 0.4194],
        [0.1935, 0.2581, 0.3226, 0.3871, 0.4516]])
--------------- weekday ----------------
tensor([[0.8333, 0.0000, 0.3333, 0.6667, 1.0000],
        [1.0000, 0.1667, 0.5000, 0.8333, 0.0000],
        [0.0000, 0.3333, 0.6667, 1.0000, 0.1667]])
---------------- weekly ----------------
tensor([[0.0000, 0.0227, 0.0227, 0.0227, 0.0227],
        [0.0000, 0.0227, 0.0227, 0.0227, 0.0455],
        [0.0227, 0.0227, 0.0227, 0.0227, 0.0455]])
--------------- monthly ----------------
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
------------------ y -------------------
tensor([14., 15., 16.])
